In [ ]:
import numpy as np
from scipy.io import mmread
# from scipy.sparse import csr_matrix # Non usata se A è densa
import time
from scipy.optimize import minimize_scalar
import pandas as pd
import matplotlib as plt
import matplotlib.ticker as mticker



In [ ]:
def projection_simplex(v, z=1.0):
    """Project vector v onto the simplex (sum(x)=z, x>=0)."""
    n = len(v)
    if n == 0: return np.array([])
    u = np.sort(v)[::-1]
    cssv = np.cumsum(u)
    
    
    rho_idx = -1
    for i in range(n - 1, -1, -1): 
        if u[i] + (z - cssv[i]) / (i + 1) > 0: 
            rho_idx = i
            break
    

    if rho_idx == -1: 
        if z == 0: return np.zeros(n)
        
        pass 
    theta = (cssv[rho_idx] - z) / (rho_idx + 1.0)
    
    proj_v = np.maximum(v - theta, 0)
    
   
    current_sum = np.sum(proj_v)
    if z > 1e-9: 
        if abs(current_sum - z) > 1e-7 * z: 
            if current_sum > 1e-9: 
                proj_v = proj_v * (z / current_sum)
            else: 
                  pass
    elif abs(current_sum) > 1e-7 : 
         proj_v = np.zeros(n)
         
    return proj_v


def lmo(grad):
    """Linear Minimization Oracle for the simplex."""
    i = np.argmin(grad)
    s = np.zeros_like(grad)
    s[i] = 1.0
    return s, i 

def f_l2(x, A):
    """L2-regularized objective function"""
    return x @ A @ x + 0.5 * np.dot(x, x)

def grad_l2(x, A):
    """Gradient for L2-regularized objective"""
    
    return 2 * (A @ x) + x

def f_l0(x, A, alpha=0.07, beta=5):
    """L0-regularized objective function"""
   
    reg_term = alpha * np.sum(np.exp(-beta * np.clip(x, 0, None)) - 1.0)
    return x @ A @ x + reg_term

def grad_l0(x, A, alpha=0.07, beta=5):
    """Gradient for L0-regularized objective"""
    return 2 * (A @ x) - alpha * beta * np.exp(-beta * np.clip(x, 0, None))


def line_search_generic(objective_func, x_curr, d_curr, A_matrix, gamma_max=1.0, reg_type_info='l2', alpha_l0=0.07, beta_l0=5):
    """Line search to find perfect stepsize."""
    if np.linalg.norm(d_curr) < 1e-12: 
        return 0.0
    
    def func_to_minimize(gamma_ls):
        x_candidate = x_curr + gamma_ls * d_curr 
        
        # Project candidate for L0 to ensure non-negativity for the exp term
        if reg_type_info == 'l0':
            x_candidate = np.maximum(x_candidate, 0)

        if reg_type_info == 'l2' :
            val = objective_func(x_candidate, A_matrix) 
        else: # CORRECTED BLOCK
            val = objective_func(x_candidate, A_matrix, alpha=alpha_l0, beta=beta_l0)
        return -val # We minimize the negative objective to maximize the objective
    
    if not (np.isfinite(gamma_max) and gamma_max >= 0): gamma_max = 1.0
    
    if gamma_max < 1e-12: return 0.0

    try:
        res = minimize_scalar(func_to_minimize, bounds=(0, gamma_max), method='bounded')
        gamma = res.x
        if not np.isfinite(gamma): gamma = 0.0
    except Exception:
        gamma = 0.0 
        
    return np.clip(gamma, 0.0, gamma_max)

def extract_clique(x, A, threshold=1e-5):
    """Extract clique from solution vector."""
    S = np.where(x > threshold)[0]
    if len(S) == 0:
        return []
    

    sorted_indices = S[np.argsort(-x[S])]
    clique = []
    for i in sorted_indices:
       
        is_connected = True
        for j in clique:
            if A[i, j] == 0:
                is_connected = False
                break
        if is_connected:
            clique.append(i)
    return clique


def load_graph(file_path):
    """Load graph from MTX file and return adjacency matrix."""
    sparse_matrix = mmread(file_path)
    A_dense = sparse_matrix.toarray()
    np.fill_diagonal(A_dense, 0) 
    return A_dense

# Algorithms

# Fank-Wolfe

In [ ]:
def frank_wolfe(A, reg_type='l2', max_iter=400, tol=1e-6, fixed_stepsize=None):
    n = A.shape[0]
    x = np.ones(n) / n 

    objective_function = f_l2 if reg_type == 'l2' else f_l0
    gradient_function = grad_l2 if reg_type == 'l2' else grad_l0

    history = [objective_function(x, A)]

    for t in range(max_iter):
        grad_orig = gradient_function(x, A)  
        grad_min = -grad_orig                   

        s_vertex, _ = lmo(grad_min)
        d = s_vertex - x

        gap = grad_orig @ d

        if gap < tol:
            
            break


        if fixed_stepsize is not None:
            gamma = fixed_stepsize
        else:
            if reg_type == 'l2':
                gamma = line_search_generic(objective_function, x, d, A, gamma_max=1.0, reg_type_info='l2')
            else: 
                gamma = line_search_generic(objective_function, x, d, A, gamma_max=1.0, reg_type_info='l0')
       
        x = x + gamma * d

        history.append(objective_function(x, A))


    return x, history


# Pairwise Frank-Wolfe

In [ ]:
def pairwise_frank_wolfe(A, reg_type='l2', max_iter=400, tol=1e-6, fixed_stepsize=None):
    n = A.shape[0]
    x0_idx = np.random.randint(n)
    x = np.zeros(n)
    x[x0_idx] = 1.0  

    objective_function = f_l2 if reg_type == 'l2' else f_l0
    gradient_function = grad_l2 if reg_type == 'l2' else grad_l0

    history = [objective_function(x, A)]

    for t in range(max_iter):
        grad_orig = gradient_function(x, A)
        grad_min = -grad_orig

        s_vertex_fw, s_index = lmo(grad_min)  

        d_for_gap = s_vertex_fw - x
        gap = grad_orig @ d_for_gap
        if gap < tol:
            break

        current_support_indices = np.where(x > 1e-9)[0]
        if len(current_support_indices) == 0:
            x.fill(0.0)
            x[s_index] = 1.0
            
            history.append(objective_function(x, A))
            continue

        v_index = current_support_indices[np.argmax(grad_orig[current_support_indices])]

        if s_index == v_index:
            d = s_vertex_fw - x 
            gamma_max_ls = 1.0
        else:
            v_vertex = np.zeros(n)
            v_vertex[v_index] = 1.0
            d = s_vertex_fw - v_vertex 
            gamma_max_ls = x[v_index]

        if fixed_stepsize is not None:
            gamma = min(fixed_stepsize, gamma_max_ls)
        else:
            gamma = line_search_generic(objective_function, x, d, A, gamma_max=gamma_max_ls, reg_type_info=reg_type)

        if s_index == v_index:
            x = (1.0 - gamma) * x + gamma * s_vertex_fw
        else:
            x[s_index] += gamma
            x[v_index] -= gamma

        x[x < 1e-10] = 0.0
        current_sum = np.sum(x)
        if abs(current_sum - 1.0) > 1e-7 and current_sum > 1e-9:
            x /= current_sum


        history.append(objective_function(x, A))


    return x, history

# Away-step Frank-Wolfe

In [ ]:
def away_step_frank_wolfe(A, reg_type='l2', max_iter=400, tol=1e-6, fixed_stepsize=None):
    n = A.shape[0]

    x0_idx = np.random.randint(n)
    x = np.zeros(n)
    x[x0_idx] = 1.0

    objective_function = f_l2 if reg_type == 'l2' else f_l0
    gradient_function = grad_l2 if reg_type == 'l2' else grad_l0

    history = [objective_function(x, A)]

    for t in range(max_iter):
        grad_orig = gradient_function(x, A)
        grad_min = -grad_orig

        s_vertex_fw, s_index = lmo(grad_min)
        d_fw = s_vertex_fw - x
        
        gap_fw = grad_orig @ d_fw
        if gap_fw < tol:
            break

        current_support_indices = np.where(x > 1e-9)[0]
        if len(current_support_indices) == 0:
            x.fill(0.0)
            x[s_index] = 1.0
            history.append(objective_function(x, A))
            continue
            
        v_index = current_support_indices[np.argmax(grad_orig[current_support_indices])]
        v_vertex = np.zeros(n)
        v_vertex[v_index] = 1.0
        d_away = x - v_vertex
        
        potential_progress_fw = grad_orig @ d_fw
        potential_progress_away = grad_orig @ d_away
        
        direction_type = ""
        if potential_progress_fw >= potential_progress_away:
            d = d_fw
            gamma_max_ls = 1.0
            direction_type = "FW"
        else:
            d = d_away
            alpha_v_t = x[v_index]
            if abs(1.0 - alpha_v_t) < 1e-9 or np.linalg.norm(d_away) < 1e-9:
                d = d_fw
                gamma_max_ls = 1.0
                direction_type = "FW (fallback from Away)"
            else:
                gamma_max_ls = alpha_v_t / (1.0 - alpha_v_t)
                direction_type = "Away"
        
        if fixed_stepsize is not None:
            gamma = min(fixed_stepsize, gamma_max_ls)
        else:
            gamma = line_search_generic(objective_function, x, d, A, gamma_max=gamma_max_ls, reg_type_info=reg_type)

        if direction_type.startswith("FW"):
            x = (1.0 - gamma) * x
            x[s_index] += gamma
        elif direction_type == "Away":
            x = (1.0 + gamma) * x
            x[v_index] -= gamma
        
        x[x < 1e-10] = 0.0
        current_sum = np.sum(x)
        if abs(current_sum - 1.0) > 1e-7 and current_sum > 1e-9:
            x /= current_sum
        

        history.append(objective_function(x, A))
            
    return x, history

# Projected Gradient

In [ ]:
def projected_gradient(A, reg_type='l2', max_iter=200, tol=1e-6, lr_initial=0.1, fixed_learning_rate=None):
    n = A.shape[0]
    x = np.ones(n) / n  

    if reg_type == 'l2':
        objective_function = f_l2
        gradient_function = grad_l2
    else:  
        objective_function = f_l0
        gradient_function = grad_l0

    history = [objective_function(x, A)]

    for t in range(max_iter):
       
        grad_orig = gradient_function(x, A)

        
        if fixed_learning_rate is not None:
            gamma = fixed_learning_rate
        else:
            gamma = line_search_generic(
                objective_func=objective_function,
                x_curr=x,
                d_curr=grad_orig, 
                A_matrix=A,
                gamma_max=lr_initial,
                reg_type_info=reg_type
            )

        x_candidate_unprojected = x + gamma * grad_orig

        x_next = projection_simplex(x_candidate_unprojected)

        if np.linalg.norm(x_next - x) < tol:
            break

        x = x_next


        history.append(objective_function(x, A))

    return x, history


# Experiments execution

In [ ]:

graph_files = ["./data/C250-9.mtx", "./data/C500-9.mtx", "./data/C2000-9.mtx"]
reg_types = ['l2', 'l0']
algorithms = {
    'FW': frank_wolfe,
    'PFW': pairwise_frank_wolfe,
    'AFW': away_step_frank_wolfe,
    'PGD': projected_gradient
}
stepsize_options = [None, 0.15]


results_list = []
for graph_file in graph_files:
    print(f"\n--- Processing graph file: {graph_file} ---")
    try:
        A = load_graph(graph_file)
    except FileNotFoundError:
        print(f"Errore: File del grafo '{graph_file}' non trovato. Salto questo file.")
        continue
    n = A.shape[0]

    
    for step in stepsize_options:
        step_label = 'Linear Search' if step is None else f'Fixed ({step})'
        print(f"--- Running experiments with stepsize: {step_label} for {graph_file} ---")
        for reg_type in reg_types:
            for algo_name, algo_func in algorithms.items():
                print(f"Running: {algo_name} - Reg: {reg_type} - Step: {step_label}")
                try:
                    start_time = time.perf_counter()

                    kwargs = {'A': A, 'reg_type': reg_type, 'max_iter': 100}
                    if algo_name == 'PGD':
                        kwargs['fixed_learning_rate'] = step 
                        if step is None:
                            kwargs['lr_initial'] = 0.1
                    else:
                        kwargs['fixed_stepsize'] = step

                    x, history = algo_func(**kwargs)

                    runtime = time.perf_counter() - start_time
                    clique = extract_clique(x, A)
                    clique_size = len(clique)

                    is_valid = True
                    if clique_size > 1:
                        is_valid = all(A[i, j] == 1 for i in clique for j in clique if i != j)

                    results_list.append({
                        'Graph File': graph_file,
                        'Algorithm': algo_name,
                        'Regularization': reg_type,
                        'Stepsize': step_label,
                        'Clique Size': clique_size,
                        'Valid Clique': is_valid,
                        'Runtime (s)': runtime,
                        'History': history
                    })
                except Exception as e:
                    print(f"Error in {algo_name} with {reg_type} and step {step_label} for {graph_file}: {e}")
                    import traceback
                    traceback.print_exc()
                    results_list.append({
                        'Graph File': graph_file,
                        'Algorithm': algo_name, 'Regularization': reg_type,
                        'Stepsize': step_label, 'Clique Size': -1,
                        'Valid Clique': False, 'Runtime (s)': -1,
                        'Error': str(e), 'History': []
                    })


print("\n" + "="*80)
print("                 FINAL RESULTS TABLE (Pandas DataFrame)                  ")
print("="*80)

if results_list:
    df_results = pd.DataFrame(results_list)


    columns_to_show = [col for col in df_results.columns if col != 'History']


    display_order = ['Graph File', 'Algorithm', 'Regularization', 'Stepsize', 'Clique Size', 'Valid Clique', 'Runtime (s)']
    if 'Error' in columns_to_show:
        display_order.append('Error')

    final_columns = [col for col in display_order if col in columns_to_show]

    final_columns.extend([col for col in columns_to_show if col not in final_columns])


    pd.set_option('display.max_rows', None)
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', 1000)
    print(df_results[final_columns].to_string())

print("="*80)


In [ ]:
# df_results.to_csv("results.csv")

# Graphs

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker

# Definisce colori e stili per una migliore leggibilità
palette = {
    'FW': ('#1f77b4', '-'),   # Blu
    'PFW': ('#ff7f0e', '--'), # Arancione
    'AFW': ('#2ca02c', ':'),   # Verde
    'PGD': ('#d62728', '-.')  # Rosso
}

# Ottiene la lista unica di file di grafo dal DataFrame
unique_graphs = df_results['Graph File'].unique()

# Itera su ogni file di grafo per creare una figura separata
for graph in unique_graphs:
    fig, axes = plt.subplots(2, 2, figsize=(18, 14), sharex=True)
    fig.suptitle(f"Analisi della Convergenza per il Grafo: {graph.split('/')[-1]}", fontsize=20, fontweight='bold')

    # Filtra il DataFrame per il grafo corrente
    df_graph = df_results[df_results['Graph File'] == graph]

    # Mappa per titoli e filtri
    plot_map = {
        (0, 0): ('Regolarizzazione L2 - Stepsize Fissa', (df_graph['Regularization'] == 'l2') & (df_graph['Stepsize'].str.contains('Fixed'))),
        (0, 1): ('Regolarizzazione L2 - Stepsize Dinamica (Linear Search)', (df_graph['Regularization'] == 'l2') & (df_graph['Stepsize'] == 'Linear Search')),
        (1, 0): ('Regolarizzazione L0 - Stepsize Fissa', (df_graph['Regularization'] == 'l0') & (df_graph['Stepsize'].str.contains('Fixed'))),
        (1, 1): ('Regolarizzazione L0 - Stepsize Dinamica (Linear Search)', (df_graph['Regularization'] == 'l0') & (df_graph['Stepsize'] == 'Linear Search'))
    }

    # Itera sulla mappa per creare ogni subplot
    for (row, col), (title, condition) in plot_map.items():
        ax = axes[row, col]
        df_subset = df_graph[condition]

        if df_subset.empty:
            ax.text(0.5, 0.5, 'Nessun dato disponibile', horizontalalignment='center', verticalalignment='center', transform=ax.transAxes)
        else:
            # Plotta la history per ogni algoritmo
            for _, result_row in df_subset.iterrows():
                algo = result_row['Algorithm']
                history = result_row['History']
                if history: # Controlla se la history non è vuota
                    color, style = palette.get(algo, ('black', '-'))
                    ax.plot(history, label=algo, color=color, linestyle=style, linewidth=2)

        # Impostazioni del subplot
        ax.set_title(title, fontsize=14, fontweight='medium')
        ax.set_ylabel("Valore Funzione Obiettivo", fontsize=12)
        ax.grid(True, which='both', linestyle='--', linewidth=0.5)
        ax.legend()
        ax.tick_params(axis='both', which='major', labelsize=10)
        ax.yaxis.set_major_formatter(mticker.FormatStrFormatter('%.2f'))


    # Impostazioni comuni per le righe di subplot
    for i in range(2):
        axes[i, 0].set_ylabel("Valore Funzione Obiettivo", fontsize=12)
    for i in range(2):
        axes[1, i].set_xlabel("Iterazioni", fontsize=12)


    plt.tight_layout(rect=[0, 0, 1, 0.96])
    # Salva la figura invece di mostrarla, per un migliore output
    # plt.savefig(f"plot_{graph.split('/')[-1].replace('.mtx', '')}.png")
    plt.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

df_valid_results = df_results[(df_results['Valid Clique'] == True) & (df_results['Clique Size'] > 0)]

if 'Error' in df_valid_results.columns:
    df_valid_results = df_valid_results[df_valid_results['Error'].isnull()]

if df_valid_results.empty:
    print("Warning: No valid results found after filtering. Plots might be empty or not generated.")
else:
    print(f"Number of valid results for plotting: {len(df_valid_results)}")
    
    plt.figure(figsize=(14, 7))
    sns.barplot(data=df_valid_results, x='Algorithm', y='Clique Size', hue='Stepsize', palette='viridis', errorbar=None)
    plt.title('Maximum Clique Size by Algorithm and Stepsize', fontsize=16)
    plt.xlabel('Algorithm', fontsize=12)
    plt.ylabel('Maximum Clique Size', fontsize=12)
    plt.xticks(rotation=45, ha='right')
    plt.legend(title='Stepsize')
    plt.tight_layout()
    plt.show()

    plt.figure(figsize=(14, 7))
    sns.barplot(data=df_valid_results, x='Algorithm', y='Runtime (s)', hue='Stepsize', palette='mako', errorbar=None)
    plt.title('Average Runtime by Algorithm and Stepsize', fontsize=16)
    plt.xlabel('Algorithm', fontsize=12)
    plt.ylabel('Average Runtime (s)', fontsize=12)
    plt.xticks(rotation=45, ha='right')
    plt.legend(title='Stepsize')
    plt.tight_layout()
    plt.show()




    grouped_clique = df_valid_results.groupby(['Algorithm', 'Regularization', 'Stepsize'])['Clique Size'].mean().reset_index()
    
    if not grouped_clique.empty:
        g = sns.catplot(data=grouped_clique, x='Algorithm', y='Clique Size', hue='Stepsize',
                        col='Regularization', kind='bar', palette='pastel', height=5, aspect=1.2)
        g.fig.suptitle('Impact of Stepsize and Regularization on Clique Size', y=1.03)
        g.set_axis_labels("Algorithm", "Average Clique Size")
        g.set_titles("Regularization: {col_name}")
        plt.tight_layout()
        plt.show()
    else:
        print("Not enough data for the 'Impact of Stepsize on Clique Size' plot.")

    plt.figure(figsize=(12, 7))
    sns.boxplot(data=df_valid_results, x='Algorithm', y='Clique Size', hue='Regularization', palette='Set2')
    plt.title('Impact of Regularization on Clique Size Distribution', fontsize=16)
    plt.xlabel('Algorithm', fontsize=12)
    plt.ylabel('Clique Size', fontsize=12)
    plt.xticks(rotation=45, ha='right')
    plt.legend(title='Regularization')
    plt.tight_layout()
    plt.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# Assicurati che le librerie necessarie siano importate.
# Il tuo DataFrame 'df_results' dovrebbe essere disponibile a questo punto.

# --- Inizio del codice per la visualizzazione ---

# 1. Filtriamo i risultati per includere solo le clique valide, per evitare di plottare dati errati.
#    Usiamo .copy() per evitare warning da parte di pandas.
df_valid_results = df_results[df_results['Valid Clique']].copy()

# Se non ci sono risultati validi, stampa un messaggio ed esci.
if df_valid_results.empty:
    print("Nessun risultato valido da visualizzare.")
else:
    # 2. Otteniamo la lista unica degli algoritmi testati
    algorithms = df_valid_results['Algorithm'].unique()
    
    # 3. Creiamo una griglia di subplot. 2 righe, 2 colonne.
    #    Aumentiamo la figsize per dare spazio a tutti i grafici.
    fig, axes = plt.subplots(2, 2, figsize=(18, 14))
    
    # "Appiattiamo" l'array di assi per iterare più facilmente con un singolo ciclo for
    axes = axes.flatten()

    # 4. Iteriamo su ogni algoritmo e sul corrispondente asse (subplot)
    for i, algo_name in enumerate(algorithms):
        ax = axes[i] # Selezioniamo l'asse corrente
        
        # Filtriamo il DataFrame per ottenere solo i dati dell'algoritmo corrente
        df_algo = df_valid_results[df_valid_results['Algorithm'] == algo_name]
        
        # Creiamo lo scatter plot sull'asse corrente
        sns.scatterplot(
            data=df_algo,
            x='Runtime (s)',
            y='Clique Size',
            hue='Regularization', # Il colore dipende dalla regolarizzazione (come richiesto)
            style='Stepsize',     # Usiamo lo stile del marker per differenziare lo stepsize
            size='Stepsize',      # Usiamo anche la dimensione per lo stepsize
            sizes=(100, 250),     # Intervallo di dimensioni per i punti
            palette='viridis',    # Un'altra palette di colori, per esempio 'viridis' o 'plasma'
            ax=ax                 # Specifichiamo su quale subplot disegnare
        )
        
        # Impostiamo il titolo e le etichette per ogni subplot
        ax.set_title(f"Performance per l'algoritmo: {algo_name}", fontsize=14, weight='bold')
        ax.set_xlabel('Runtime (s)', fontsize=12)
        ax.set_ylabel('Clique Size', fontsize=12)
        ax.grid(True, linestyle='--', alpha=0.6)
        ax.legend(title='Legenda') # Mostra la legenda su ogni subplot

    # 5. Se il numero di algoritmi non è un multiplo di 2 (es. 3), nascondiamo gli assi vuoti.
    for i in range(len(algorithms), len(axes)):
        fig.delaxes(axes[i])

    # 6. Aggiungiamo un titolo principale per l'intera figura
    fig.suptitle('Confronto Performance: Dimensione Clique vs. Runtime per Algoritmo', fontsize=20, y=0.98)

    # 7. Ottimizziamo il layout per evitare sovrapposizioni di titoli ed etichette
    plt.tight_layout(rect=[0, 0, 1, 0.96]) # rect lascia spazio per il suptitle

    # 8. Mostriamo il grafico
    plt.show()

In [ ]:
# ======================================================================
# SCRIPT PER L'ANALISI DI ALGORITMI DI OTTIMIZZAZIONE
# SUL PROBLEMA DELLA MASSIMA CLIQUE REGOLARIZZATA
#
# Include:
# - Algoritmi: Frank-Wolfe, Pairwise FW, Away-Step FW, Projected Gradient
# - Strategie di passo: Ricerca Lineare Esatta, Regola di Armijo
# - Problemi: Regolarizzazione L2 e L0-approssimata
# ======================================================================

import numpy as np
import pandas as pd
from scipy.optimize import minimize_scalar
import time
from scipy.io import mmread

# ======================================================================
# --- 1. FUNZIONI AUSILIARIE ---
# ======================================================================

def lmo(grad):
    i = np.argmin(grad)
    s = np.zeros_like(grad)
    s[i] = 1.0
    return s, i

def projection_simplex(v, z=1):
    n_features = v.shape[0]
    
    u = np.sort(v)[::-1]
    cssv = np.cumsum(u) - z
    ind = np.arange(n_features) + 1
    cond = u * ind > cssv
    
    if np.sum(cond) > 0:
        rho = ind[cond][-1]
        theta = cssv[cond][-1] / float(rho)
        w = np.maximum(v - theta, 0)
    else:
        w = np.zeros_like(v)
        if v.size > 0: w[np.argmax(v)] = z
            
    return w

def extract_clique(x, A):
    if x is None or not np.all(np.isfinite(x)):
        return []
    
    support_indices = np.where(x > 1e-3)[0]
    if len(support_indices) == 0: 
        return []
    
    # Ordina gli indici dei nodi per peso decrescente
    sorted_support = support_indices[np.argsort(-x[support_indices])]
    
    clique = []
    for i in sorted_support:
        # Verifica se il nodo i è connesso a tutti i nodi già nella cricca
        is_connected_to_all = all(A[i, j] == 1 for j in clique)
        if is_connected_to_all:
            clique.append(i)
    return clique

def validate_clique(clique, A):
    """Verifica se un dato insieme di nodi forma una cricca."""
    if len(clique) <= 1: 
        return True

    subgraph = A[np.ix_(clique, clique)]
    # In una cricca, tutti gli elementi fuori diagonale devono essere 1.
    # La somma deve essere il numero di archi in un grafo completo.
    expected_edges = len(clique) * (len(clique) - 1)
    return np.sum(subgraph) == expected_edges


def load_graph(filepath):
    """Carica un grafo da file .mtx e restituisce la sua matrice di adiacenza."""
    sparse_matrix = mmread(filepath)

    n = sparse_matrix.shape[0]
    A = sparse_matrix.astype(int).toarray()
    
    # Assicura che sia una matrice di adiacenza 0/1, simmetrica, con diagonale nulla
    A = np.ceil((A + A.T) / 2.0).astype(int)
    np.fill_diagonal(A, 0)
    return A


# ======================================================================
# --- 2. FUNZIONI OBIETTIVO E GRADIENTE (per Massimizzazione) ---
# ======================================================================

def f_l2(x, A, alpha=0.5): 
    return x.T @ A @ x + alpha * np.dot(x, x)

def grad_l2(x, A, alpha=0.5): 
    return 2 * A @ x + alpha * 2 * x

def f_l0(x, A, alpha=0.07, beta=5): 
    return x.T @ A @ x + alpha * np.sum(np.exp(-beta * x) - 1)

def grad_l0(x, A, alpha=0.07, beta=5): 
    return 2 * A @ x - alpha * beta * np.exp(-beta * x)

# ======================================================================
# --- 3. STRATEGIE DI STEP SIZE ---
# ======================================================================

def exact_line_search(objective_func, x_curr, d_curr, A_matrix, grad_f_x, gamma_max=1.0, **obj_kwargs):
    """Ricerca lineare esatta ("Optimal Gamma")."""
    if np.linalg.norm(d_curr) < 1e-12: 
        return 0.0
    
    def func_to_minimize(gamma_ls):
        val = objective_func(x_curr + gamma_ls * d_curr, A_matrix, **obj_kwargs)
        return -val # Minimizza il negativo per massimizzare l'obiettivo
        
    if not (np.isfinite(gamma_max) and gamma_max >= 0):
        gamma_max = 1.0

    if gamma_max < 1e-12:
        return 0.0
    
    try:
        res = minimize_scalar(func_to_minimize, bounds=(0, gamma_max), method='bounded')
        gamma = res.x
        if not np.isfinite(gamma): 
            gamma = 0.0
    except Exception:
        gamma = 0.0
        
    return np.clip(gamma, 0.0, gamma_max)

def armijo_line_search(objective_func, x_curr, d_curr, A_matrix, grad_f_x, gamma_max=1.0, **obj_kwargs):
    """Ricerca lineare inesatta con regola di Armijo e backtracking."""
    if np.linalg.norm(d_curr) < 1e-12: return 0.0
    
    gamma = gamma_max  # Inizia con il passo massimo possibile
    c1 = 1e-4          # Parametro standard di Armijo
    tau = 0.5          # Fattore di riduzione del backtracking
    
    f_x = objective_func(x_curr, A_matrix, **obj_kwargs)
    progress_expected = np.dot(grad_f_x, d_curr)

    # La direzione deve essere di salita per la massimizzazione
    if progress_expected <= 0: return 0.0 

    for _ in range(20): # Limita le iterazioni del backtracking
        f_x_new = objective_func(x_curr + gamma * d_curr, A_matrix, **obj_kwargs)
        # Condizione di Armijo per la massimizzazione
        if f_x_new >= f_x + c1 * gamma * progress_expected:
            return gamma
        gamma *= tau
        
    return 0.0 # Il backtracking non ha trovato un passo valido

# ======================================================================
# --- 4. ALGORITMI DI OTTIMIZZAZIONE ---
# ======================================================================

def frank_wolfe(A, reg_type, max_iter, tol, step_func):
    """Implementazione del Frank-Wolfe Classico."""
    n = A.shape[0]
    # FW parte spesso dal baricentro per una migliore esplorazione iniziale
    x = np.ones(n) / n
    
    # Seleziona le funzioni e i parametri corretti
    if reg_type == 'l2':
        obj_func, grad_func, obj_kwargs = f_l2, grad_l2, {'alpha': 0.5}
    else:
        obj_func, grad_func, obj_kwargs = f_l0, grad_l0, {'alpha': 0.07, 'beta': 5}
    
    history = [obj_func(x, A, **obj_kwargs)]
    x_best = np.copy(x)
    f_best = history[0]
    
    for t in range(max_iter):
        grad = grad_func(x, A, **obj_kwargs)
        s_vertex, _ = lmo(-grad)
        d = s_vertex - x
        gap = np.dot(grad, d)
        if gap < tol: break
        
        gamma = step_func(obj_func, x, d, A, grad, 1.0, **obj_kwargs)
        x = (1 - gamma) * x + gamma * s_vertex

        current_f = obj_func(x, A, **obj_kwargs)
        history.append(current_f)
        if current_f > f_best:
            f_best = current_f
            x_best = np.copy(x)
            
    return x_best, history

def pairwise_frank_wolfe(A, reg_type, max_iter, tol, step_func):
    """Implementazione Corretta del Pairwise Frank-Wolfe."""
    n = A.shape[0]

    x0_idx = np.random.randint(n)
    x = np.zeros(n)
    x[x0_idx] = 1.0
    
    if reg_type == 'l2':
        obj_func = f_l2
        grad_func = grad_l2
        obj_kwargs = {'alpha': 0.5}
    else:
        obj_func = f_l0
        grad_func = grad_l0
        obj_kwargs = {'alpha': 0.07, 'beta': 5}
        
    history = [obj_func(x, A, **obj_kwargs)]
    x_best = np.copy(x)
    f_best = history[0]
    
    for t in range(max_iter):
        grad = grad_func(x, A, **obj_kwargs)
        s_vertex, s_idx = lmo(-grad)
        d_gap_vec = s_vertex - x
        gap = np.dot(grad, d_gap_vec)
        if gap < tol:
            break
        
        support = np.where(x > 1e-9)[0]
        if not len(support):
            x.fill(0.0); x[s_idx] = 1.0
            history.append(obj_func(x, A, **obj_kwargs))
            continue
            
        v_idx = support[np.argmin(grad[support])]
        
        if s_idx == v_idx:
            d = s_vertex - x
            gamma_max = 1.0
        else:
            v_vertex = np.zeros(n); v_vertex[v_idx] = 1.0
            d = s_vertex - v_vertex
            gamma_max = x[v_idx]
        
        gamma = step_func(obj_func, x, d, A, grad, gamma_max, **obj_kwargs)
        
        if s_idx == v_idx:
            x = (1.0 - gamma) * x + gamma * s_vertex
        else:
            x[s_idx] += gamma
            x[v_idx] -= gamma
        
        x[x<1e-10] = 0.0
        if np.sum(x) > 1e-9:
            x /= np.sum(x)
            
        current_f = obj_func(x, A, **obj_kwargs)
        history.append(current_f)
        if current_f > f_best:
            f_best = current_f
            x_best = np.copy(x)

    return x_best, history

def away_step_frank_wolfe(A, reg_type, max_iter, tol, step_func):
    """Implementazione Corretta dell'Away-Step Frank-Wolfe."""
    n = A.shape[0]
    
    x0_idx = np.random.randint(n)
    x = np.zeros(n)
    x[x0_idx] = 1.0
    
    if reg_type == 'l2':
        obj_func = f_l2
        grad_func = grad_l2
        obj_kwargs = {'alpha': 0.5}
    else:
        obj_func = f_l0
        grad_func = grad_l0
        obj_kwargs = {'alpha': 0.07, 'beta': 5}
        
    history = [obj_func(x, A, **obj_kwargs)]
    x_best = np.copy(x)
    f_best = history[0]
    
    for t in range(max_iter):
        grad = grad_func(x, A, **obj_kwargs)
        s_vertex, s_idx = lmo(-grad)
        d_fw = s_vertex - x
        gap_fw = np.dot(grad, d_fw)
        if gap_fw < tol: break
        
        support = np.where(x > 1e-9)[0]
        if not len(support):
            x.fill(0.0); x[s_idx] = 1.0
            history.append(obj_func(x, A, **obj_kwargs))
            continue
            
        v_idx = support[np.argmin(grad[support])]
        v_vertex = np.zeros(n); v_vertex[v_idx] = 1.0
        d_away = x - v_vertex
        progress_away = np.dot(grad, d_away)
        
        if gap_fw >= progress_away:
            d = d_fw
            gamma_max = 1.0
        else:
            d = d_away
            alpha_v = x[v_idx]
            gamma_max = alpha_v / (1.0 - alpha_v) if abs(1.0 - alpha_v) > 1e-9 else float('inf')
        
        gamma = step_func(obj_func, x, d, A, grad, gamma_max, **obj_kwargs)
        
        # L'aggiornamento per AFW è x_new = x + gamma*d
        x = x + gamma * d
        
        x[x<1e-10] = 0.0
        if np.sum(x) > 1e-9:
            x /= np.sum(x)
            
        current_f = obj_func(x, A, **obj_kwargs)
        history.append(current_f)
        if current_f > f_best:
            f_best = current_f
            x_best = np.copy(x)
    
    return x_best, history

def projected_gradient(A, reg_type, max_iter, tol, step_func, lr_initial=0.001):
    """Implementazione del Gradiente Proiettato."""
    n = A.shape[0]; x = np.ones(n) / n
    
    if reg_type == 'l2':
        obj_func = f_l2
        grad_func = grad_l2
        obj_kwargs = {'alpha': 0.5}
    else:
        obj_func = f_l0
        grad_func = grad_l0
        obj_kwargs = {'alpha': 0.07, 'beta': 5}
        
    history = [obj_func(x, A, **obj_kwargs)]
    x_best = np.copy(x)
    f_best = history[0]
    
    for t in range(max_iter):
        grad = grad_func(x, A, **obj_kwargs)
        
        # Per PGD la direzione è il gradiente. Il passo Armijo è una scelta valida.
        gamma = step_func(obj_func, x, grad, A, grad, lr_initial, **obj_kwargs)
        
        x_next = projection_simplex(x + gamma * grad)
        if np.linalg.norm(x_next - x) < tol: break
        x = x_next
        
        current_f = obj_func(x, A, **obj_kwargs)
        history.append(current_f)
        if current_f > f_best:
            f_best = current_f
            x_best = np.copy(x)
            
    return x_best, history

# ======================================================================
# --- BLOCCO DI ESECUZIONE PRINCIPALE ---
# ======================================================================
if __name__ == '__main__':
    # Aggiorna con i percorsi corretti ai tuoi file .mtx
    graph_files = [
        "./data/C250-9.mtx",
        "./data/C500-9.mtx",
        "./data/C2000-9.mtx"
    ]
    reg_types = ['l2', 'l0']
    algorithms = {
        'FW': frank_wolfe,
        'PFW': pairwise_frank_wolfe,
        'AFW': away_step_frank_wolfe,
        'PGD': projected_gradient
    }
    
    # OPZIONI PER LO STEPSIZE: Ricerca Esatta (Optimal) vs. Armijo (Inesatta)
    stepsize_options = [
        {'label': 'Optimal Gamma (LS)', 'func': exact_line_search},
        {'label': 'Armijo Rule', 'func': armijo_line_search}
    ]

    results_list = []
    
    for graph_file in graph_files:
        print(f"\n--- Elaborazione del file: {graph_file} ---")
        A = load_graph(graph_file)
        if A is None: continue
        print(f"Grafo caricato con {A.shape[0]} nodi.")
        
        # Il multi-start è importante per i problemi non convessi
        num_random_starts = 5 # Aumenta per un'analisi più robusta
        
        for step_info in stepsize_options:
            step_func = step_info['func']
            step_label = step_info['label']
            
            print(f"--- Esecuzione con strategia di passo: {step_label} ---")
            for reg_type in reg_types:
                for algo_name, algo_func in algorithms.items():
                    print(f"Esecuzione: {algo_name} - Reg: {reg_type}")
                    
                    best_clique_size_for_algo = 0
                    best_x_for_algo = None
                    total_runtime = 0
                    
                    # FW e PGD partono dal centro, non necessitano di multi-start in questo setup
                    starts_to_run = num_random_starts if algo_name in ['PFW', 'AFW'] else 1
                    
                    for i in range(starts_to_run):
                        try:
                            start_time = time.perf_counter()
                            
                            x_best, history = algo_func(A=A, reg_type=reg_type, max_iter=100, tol=1e-5, step_func=step_func)
                            
                            runtime = time.perf_counter() - start_time
                            total_runtime += runtime
                            
                            clique = extract_clique(x_best, A)
                            if len(clique) > best_clique_size_for_algo:
                                if validate_clique(clique, A):
                                    best_clique_size_for_algo = len(clique)
                                    best_x_for_algo = x_best # Salva il vettore soluzione migliore
                        
                        except Exception as e:
                            print(f"ERRORE durante l'esecuzione di {algo_name}: {e}")

                    # Salva il risultato migliore del multi-start
                    if best_x_for_algo is not None:
                         clique = extract_clique(best_x_for_algo, A)
                         is_valid = validate_clique(clique, A)
                         results_list.append({
                            'Graph File': graph_file.split('/')[-1], 'Algorithm': algo_name, 'Reg': reg_type,
                            'Stepsize': step_label, 'Clique Size': len(clique), 'Valid': is_valid,
                            'Runtime (s)': total_runtime / starts_to_run,
                         })
                    else: # Se nessun run ha prodotto una soluzione valida
                         results_list.append({
                            'Graph File': graph_file.split('/')[-1], 'Algorithm': algo_name, 'Reg': reg_type,
                            'Stepsize': step_label, 'Clique Size': 0, 'Valid': False,
                            'Runtime (s)': total_runtime / starts_to_run,
                         })


    # Stampa riepilogo tabellare
    print("\n" + "="*85)
    print("                              RIEPILOGO FINALE DEI RISULTATI                              ")
    print("="*85)
    if results_list:
        df_results = pd.DataFrame(results_list)
        print(df_results.to_string())
    else:
        print("Nessun risultato da visualizzare.")
    print("="*85)


--- Elaborazione del file: ./data/C250-9.mtx ---
Grafo caricato con 250 nodi.
--- Esecuzione con strategia di passo: Optimal Gamma (LS) ---
Esecuzione: FW - Reg: l2
Esecuzione: PFW - Reg: l2
Esecuzione: AFW - Reg: l2
Esecuzione: PGD - Reg: l2
Esecuzione: FW - Reg: l0
Esecuzione: PFW - Reg: l0
Esecuzione: AFW - Reg: l0
Esecuzione: PGD - Reg: l0
--- Esecuzione con strategia di passo: Armijo Rule ---
Esecuzione: FW - Reg: l2
Esecuzione: PFW - Reg: l2
Esecuzione: AFW - Reg: l2
Esecuzione: PGD - Reg: l2
Esecuzione: FW - Reg: l0
Esecuzione: PFW - Reg: l0
Esecuzione: AFW - Reg: l0
Esecuzione: PGD - Reg: l0

--- Elaborazione del file: ./data/C500-9.mtx ---
Grafo caricato con 500 nodi.
--- Esecuzione con strategia di passo: Optimal Gamma (LS) ---
Esecuzione: FW - Reg: l2
Esecuzione: PFW - Reg: l2
Esecuzione: AFW - Reg: l2
Esecuzione: PGD - Reg: l2
Esecuzione: FW - Reg: l0
Esecuzione: PFW - Reg: l0
Esecuzione: AFW - Reg: l0
Esecuzione: PGD - Reg: l0
--- Esecuzione con strategia di passo: Armijo